# Cálculo do MAE, RMSE e MAPE para todos os modelos testados

O cálculo foi feito usando os modelos com seus parâmetros padrões, sem ajustes manuais.

O MAPE não é uma estimativa confiável nesse caso, pois os dados transformados pela diferença logarítmica são muito próximos de zero.

In [35]:
from model.ar_ma_models import ARModel, MAModel
from model.baseline_models import NaiveForecast, MovingAverage
from model.exp_smooth_models import (
    SimpleExponentialSmoothingModel, 
    SimpleExponentialSmoothingModel, 
    HoltLinearTrendModel, 
    HoltWintersAdditiveModel, 
    HoltWintersMultiplicativeModel)
from model.diagnostics import WalkForwardDiagnostics
from model.data_loader import CSVDataLoader
from model.preprocessor import LogDiffTransform
import warnings
warnings.filterwarnings('ignore')

In [36]:
data_loader = CSVDataLoader("data/data_updated.csv", target_column="volume")
transformer = LogDiffTransform()


volume = transformer.transform(data_loader.get_series("volume"))
users = transformer.transform(data_loader.get_series("users"))
inv = transformer.transform(data_loader.get_series("inv"))

In [59]:
base_model_results = {}

base_models = [
    NaiveForecast(),
    MovingAverage()
]

default_model_results = {}
default_models = [
    ARModel(),
    MAModel(),
    SimpleExponentialSmoothingModel(),
    HoltLinearTrendModel(),
]

all_model_results = {}
all_models = base_models + default_models

diagnostics = WalkForwardDiagnostics(50, horizon=12)

In [60]:

for model in base_models:
    results = diagnostics.diagnose(model, volume)
    base_model_results[model.name] = results
    all_model_results[model.name] = results

base_model_results

{'NaiveForecast': {'MAE': 0.20922612298455612,
  'RMSE': 0.255863850555983,
  'MAPE': 286.5461125330522},
 'MovingAverage': {'MAE': 0.18300850497172955,
  'RMSE': 0.22082591995804127,
  'MAPE': 195.38738572805764}}

In [61]:
for model in default_models:
    results = diagnostics.diagnose(model, volume)
    default_model_results[model.name] = results
    all_model_results[model.name] = results
    
default_model_results

{'AR(1)': {'MAE': 0.1550837192250185,
  'RMSE': 0.1897358139697657,
  'MAPE': 109.46466402978487},
 'MA(1)': {'MAE': 0.15593659706015864,
  'RMSE': 0.19054234022837696,
  'MAPE': 110.20582753756167},
 'Simple Exponential Smoothing': {'MAE': 0.1586593756248776,
  'RMSE': 0.193251704367271,
  'MAPE': 126.56125317632659},
 'Holt Linear Trend': {'MAE': 0.36124582894087676,
  'RMSE': 0.47255834211322717,
  'MAPE': 632.3827535400741}}

# Considerando todas as variáveis exógenas disponíveis no dataset

In [62]:
import numpy as np
full_exog = np.column_stack((users, inv))

exog_models_results = {}
exog_models = [
    ARModel(name="AR_exog"),
    MAModel(name="MA_exog"),
]
all_models += exog_models


for model in exog_models:
    results = diagnostics.diagnose(model, volume, fit_params={"exog": full_exog}, predict_params={"exog": full_exog})
    exog_models_results[model.name] = results
    all_model_results[model.name] = results
    
exog_models_results

{'AR_exog': {'MAE': 0.16271028710322202,
  'RMSE': 0.2488985862434992,
  'MAPE': 335.41942405509445},
 'MA_exog': {'MAE': 0.13531769413577757,
  'RMSE': 0.19439309475494726,
  'MAPE': 229.52365528241515}}

In [63]:
import pandas as pd

results_df = pd.DataFrame.from_dict(all_model_results, orient='index').sort_values(by='RMSE')
results_df

,MAE,RMSE,MAPE
AR(1),0.155084,0.189736,109.464664
MA(1),0.155937,0.190542,110.205828
Simple Exponential Smoothing,0.158659,0.193252,126.561253
MA_exog,0.135318,0.194393,229.523655
MovingAverage,0.183009,0.220826,195.387386
AR_exog,0.162710,0.248899,335.419424
NaiveForecast,0.209226,0.255864,286.546113
Holt Linear Trend,0.361246,0.472558,632.382754


In [64]:
all_models[-1].get_params()

{'params': array([-0.45690659]),
 'const': 0.020904813498624844,
 'aic': -51.310141921597896,
 'bic': -36.53100663359159}

In [66]:
all_models[2].get_params()

{'params': array([-0.18908373]),
 'const': 0.027855166075948547,
 'aic': 34.07441777156474,
 'bic': 42.96295166234446}